In [1]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [3]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://previous-runs-api.open-meteo.com/v1/forecast"
params = {
	"latitude": 51.3167,
	"longitude": 9.5,
	"hourly": ["temperature_2m", "temperature_2m_previous_day1", "temperature_2m_previous_day2", "temperature_2m_previous_day3", "temperature_2m_previous_day4", "temperature_2m_previous_day5", "temperature_2m_previous_day6", "temperature_2m_previous_day7", "relative_humidity_2m", "relative_humidity_2m_previous_day1", "relative_humidity_2m_previous_day2", "relative_humidity_2m_previous_day3", "relative_humidity_2m_previous_day4", "relative_humidity_2m_previous_day5", "relative_humidity_2m_previous_day6", "relative_humidity_2m_previous_day7", "precipitation", "precipitation_previous_day1", "precipitation_previous_day2", "precipitation_previous_day3", "precipitation_previous_day4", "precipitation_previous_day5", "precipitation_previous_day6", "precipitation_previous_day7", "surface_pressure", "surface_pressure_previous_day1", "surface_pressure_previous_day2", "surface_pressure_previous_day3", "surface_pressure_previous_day4", "surface_pressure_previous_day5", "surface_pressure_previous_day7", "shortwave_radiation", "shortwave_radiation_previous_day1", "shortwave_radiation_previous_day2", "shortwave_radiation_previous_day3", "shortwave_radiation_previous_day4", "shortwave_radiation_previous_day5", "shortwave_radiation_previous_day6", "shortwave_radiation_previous_day7", "wind_speed_80m", "wind_speed_80m_previous_day1", "wind_speed_80m_previous_day2", "wind_speed_80m_previous_day3", "wind_speed_80m_previous_day4", "wind_speed_80m_previous_day5", "wind_speed_80m_previous_day6", "wind_speed_80m_previous_day7"],
	"timezone": "Europe/Berlin",
	"start_date": "2018-05-05",
	"end_date": "2024-10-30"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()

hourly_temperature_2m_previous_day1 = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m_previous_day1 = hourly.Variables(1).ValuesAsNumpy()
hourly_precipitation_previous_day1 = hourly.Variables(2).ValuesAsNumpy()
hourly_surface_pressure_previous_day1 = hourly.Variables(3).ValuesAsNumpy()
hourly_shortwave_radiation_previous_day1 = hourly.Variables(4).ValuesAsNumpy()
hourly_wind_speed_80m_previous_day1 = hourly.Variables(5).ValuesAsNumpy()

# hourly_temperature_2m = hourly.Variables(6).ValuesAsNumpy()
# hourly_relative_humidity_2m = hourly.Variables(7).ValuesAsNumpy()
# hourly_precipitation = hourly.Variables(8).ValuesAsNumpy()
# hourly_surface_pressure = hourly.Variables(9).ValuesAsNumpy()
# hourly_shortwave_radiation = hourly.Variables(10).ValuesAsNumpy()
# hourly_wind_speed_80m = hourly.Variables(1).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
# hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["temperature_2m_previous_day1"] = hourly_temperature_2m_previous_day1
# hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["relative_humidity_2m_previous_day1"] = hourly_relative_humidity_2m_previous_day1
# hourly_data["precipitation"] = hourly_precipitation
hourly_data["precipitation_previous_day1"] = hourly_precipitation_previous_day1
# hourly_data["surface_pressure"] = hourly_surface_pressure
hourly_data["surface_pressure_previous_day1"] = hourly_surface_pressure_previous_day1
# hourly_data["shortwave_radiation"] = hourly_shortwave_radiation
hourly_data["shortwave_radiation_previous_day1"] = hourly_shortwave_radiation_previous_day1
# hourly_data["wind_speed_80m"] = hourly_wind_speed_80m
hourly_data["wind_speed_80m_previous_day1"] = hourly_wind_speed_80m_previous_day1

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

csv_file = "../data/weather_forcast_prediciton.csv"
hourly_dataframe.to_csv(csv_file, index = False)

Coordinates 51.31999969482422°N 9.5°E
Elevation 162.0 m asl
Timezone b'Europe/Berlin' b'CET'
Timezone difference to GMT+0 3600 s
                           date  temperature_2m_previous_day1  \
0     2018-05-04 23:00:00+00:00                           NaN   
1     2018-05-05 00:00:00+00:00                           NaN   
2     2018-05-05 01:00:00+00:00                           NaN   
3     2018-05-05 02:00:00+00:00                           NaN   
4     2018-05-05 03:00:00+00:00                           NaN   
...                         ...                           ...   
56899 2024-10-30 18:00:00+00:00                     12.441000   
56900 2024-10-30 19:00:00+00:00                     11.990999   
56901 2024-10-30 20:00:00+00:00                     11.490999   
56902 2024-10-30 21:00:00+00:00                     11.490999   
56903 2024-10-30 22:00:00+00:00                     11.490999   

       relative_humidity_2m_previous_day1  precipitation_previous_day1  \
0               